In [9]:
from pymongo import MongoClient
import pandas as pd
import os

In [5]:
client = MongoClient('localhost', 27017)

In [6]:
db = client.publicru_test

In [7]:
collection = db.documents_collection

In [15]:
result = collection.delete_many({})
result.deleted_count

2478

In [10]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
models_path = os.path.join("..", "..", "models")
experiments_path = os.path.join("..", "..", "experiments")

In [11]:
df_processed_documents =  pd.read_csv(os.path.join(raw_data_path, "publicru-dataset-2020-03-16", "lib_public_document.csv"), index_col="id")

In [12]:
df_processed_documents["doc_id"] = df_processed_documents.index

In [13]:
df_processed_documents.head()

,content_api_id,title,annotation,body,pages,authors,size,last_modified,issue_id,pages_visual,doc_id
id,,,,,,,,,,,
325,182582609,Эффект слабого рубля,Мы взяли две компании с сопоставимой выручкой ...,<body><p><b>Мы взяли две компании с сопоставим...,"[70, 71]",NaN,40754,2016-11-24 13:15:42.057148+00:00,9,"['70', '71']",325
21225,175907949,РОССТАТ,Индексы потребительских цен на товары и услуги...,<body><p><b>Индексы потребительских цен на тов...,[4],NaN,28483,2016-11-24 23:46:59.007647+00:00,599,['4'],21225
21226,175907948,Неотложка с планшетом,ЗДОРОВЬЕ . В системе ЕМИАС зарегистрировались ...,<body><p><b>ЗДОРОВЬЕ</b> </p>\n<p>В системе ЕМ...,[4],Сергей Жуков,30346,2016-11-24 23:46:59.090845+00:00,599,['4'],21226
1638,183704356,ПРОГНОЗЫ НОМЕРА,К 2017-2018 годам ныне растущий агрокомплекс с...,<body><p>К 2017-2018 годам ныне растущий агрок...,[5],NaN,818,2016-11-24 14:16:32.322323+00:00,42,['5'],1638
2,154615279,ТАНЕЦ НЕНАСТОЯЩЕГО ЛЕБЕДЯ,ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА: ДВА МИСТ...,<body><p>ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА:...,"[60, 61]",NaN,44550,2016-11-24 13:01:38.412930+00:00,1,"['60', '61']",2


In [14]:
df_processed_documents.columns.tolist()

['content_api_id',
 'title',
 'annotation',
 'body',
 'pages',
 'authors',
 'size',
 'last_modified',
 'issue_id',
 'pages_visual',
 'doc_id']

In [18]:
%%time

posts = [ {
    "doc_id": row["doc_id"],
    "title": row["title"],
    "body": row["body"],
    "annotation": row["annotation"],
    "authors": row["authors"],
    "issue_id": row["issue_id"],
}  for _, row in df_processed_documents.iterrows()]

CPU times: user 1min 27s, sys: 254 ms, total: 1min 27s
Wall time: 1min 27s


In [19]:
%%time

collection.insert_many(posts)

CPU times: user 21.8 s, sys: 5.45 s, total: 27.3 s
Wall time: 53.4 s


In [20]:
collection.count_documents({})

582052

In [39]:
tags = collection.distinct("issue_id")

In [41]:
len(tags)

19448